## Imports

In [1]:
from geopy.geocoders import GoogleV3
import pandas as pd
import numpy as np
import os

## Read-in

In [3]:
df = pd.read_csv('Hudson_yards_map_template_popup_pin - Sheet1.csv')

In [5]:
df.columns

Index(['full_address', 'sq_ft', 'developers', 'description', 'story_link',
       'project name', 'floors', 'status', 'notable tenants'],
      dtype='object')

In [4]:
df

,full_address,sq_ft,developers,description,story_link,project name,floors,status,notable tenants
0,501 West 30th Street,1.8 million,Related Companies/Oxford Properties,Office building,NaN,10 Hudson Yards,52,Open,"Tapestry, SAP, L'Oreal USA"
1,500 West 33rd Street,2.6 million,Related Companies/Oxford Properties,Office building,NaN,30 Hudson Yards,100,Open,"Warner Media, Wells Fargo, KKR"
2,427 10th Avenue,2.9 million,Related Companies/Oxford Properties,Office building,NaN,50 Hudson Yards,77,Open,"BlackRock, Meta, Truist Financial"
3,380 11th Avenue,1.3 million,Related Companies/Oxford Properties,Office building,NaN,55 Hudson Yards,51,Open,"Meta, Point72, Cooley"
4,514 West 36th Street,1.3 million,Related Companies/Oxford Properties,Office building,NaN,NaN,25 tentatively,In Development,NaN
5,400 11th Avenue,1.9 million,Boston Properties/The Moinian Group,Office building,NaN,3 Hudson Boulevard,56,In Development,NaN
6,527 West 34th Street,2.8 million,Tishman Speyer,Office building,NaN,The Spiral,66,In Development,"HSBC, Pfizer, NewYork Presbyterian"
7,395 9th Avenue,2.1 million,Brookfield Properties,Office building,NaN,One Manhattan West,67,Open,"EY, Skadden, NHL"
8,435 West 31st Street,2.0 million,Brookfield Properties,Office building,NaN,Two Manhattan West,58,In Development,"D.E. Shaw, KPMG, Clifford Chance"
9,424/434 West 33rd Street,"184,000",Brookfield Properties,Office building,NaN,Four Manhattan West,13,Open,NaN


## Geocoder Setup

In [6]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [7]:
df['geo_address'] = df['full_address'] + ' Manhattan, NY'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

## HTML Popup Formatter

In [8]:
df.columns

Index(['full_address', 'sq_ft', 'developers', 'description', 'story_link',
       'project name', 'floors', 'status', 'notable tenants', 'geo_address',
       'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [25]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developers = df['developers'].iloc[i]
    description = df['description'].iloc[i]
    project_name = df['project name'].iloc[i]
    floors = df['floors'].iloc[i]
    status = df['status'].iloc[i]
    notable_tenants = df['notable tenants'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer: <strong>{}'''.format(developers) + '''</strong><br>
    Description: <strong>{}'''.format(description) + '''</strong><br>
    Project Name: <strong>{}'''.format(project_name) + '''</strong><br>
    Floors: <strong>{}'''.format(floors) + '''</strong><br>
    Status: <strong>{}'''.format(status) + '''</strong><br>
    Notable Tenants: <strong>{}'''.format(notable_tenants) + '''</strong><br>
    </html>
    '''
    return html


In [27]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Hudson Yards Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=300, max_width=300)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [ ]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [28]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [29]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/hudson_yards_map_Pat_Ralph
